In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install numpy 
!pip install pandas
!pip install json

In [ ]:
#Import für Datenmanipulation
import numpy as np #Für Matrizen
import pandas as pd #Für Dataframes
import re
from typing import List, Any, Set, Dict, Tuple, Optional
import json # load json module

In [ ]:
TEXT_COLUMN = 'Text'
SNOMED_CT_CODE_COLUMN = 'Code'
LABEL_COLUMN = 'Label'
LABEL_CODE_COLUMN = 'Label Code'
OHE_COLUMN = 'One Hot Encoding'


  **Daten aus Snomed CT release Package generieren**
  



Tabelle aus SNOMED CT Releasepackage erstellen

In [ ]:
# Install medcat (Quelle: https://colab.research.google.com/github/CogStack/MedCATtutorials/blob/main/notebooks/specialised/Preprocessing_SNOMED_CT.ipynb#scrollTo=CqUmB-ZH13zB)
!pip install --upgrade medcat
from medcat.utils.preprocess_snomed import Snomed

# Load local zip folder into colab repository
from google.colab import files
uploaded = files.upload()

snomed_path = 'SnomedCT_InternationalRF2_PRODUCTION_20220228T120000Z.zip' 
!unzip snomed_path

# Initialise
snomed_filename = "SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z"  # The unzippedSNOMED CT folder
snomed = Snomed(snomed_filename)

df = snomed.to_concept()

# Structure df is cui [0], name [1], name_status [2], ontologies [3] description_type_ids [4], type_ids [5]

SNOMED CT Tabelle einlesen, Typs: procedure, body structure (SNOMED CT) und disorder (NOT SNOMED CT) aus Tabelle extrahieren. Neues xlsx erstellen

In [ ]:
import csv
import pandas as pd
from openpyxl import Workbook

df = pd.read_csv(r'/content/drive/MyDrive/preprocessed_snomed.csv' , sep = ",")  # read the csv file

# Structure df is cui [0], name [1], name_status [2], ontologies [3] description_type_ids [4], type_ids [5]
text = []   
codeIDs = []
label = []

for x in range(len(df)):
  y = df['type_ids'].values[x]
  if y == 28321150 or y == 37552161:

    text.append(df['name'].values[x])
    codeIDs.append(df['cui'].values[x])
    label.append("SNOMED CT")

  elif y == 9090192:
    text.append(df['name'].values[x])
    codeIDs.append(df['cui'].values[x])
    label.append("NOT SNOMED CT")

new_df = pd.DataFrame({'Text': text,'Code': codeIDs, 'Label': label})
new_df.to_csv(r'/content/drive/MyDrive/snomed_train_df.csv', index=False)

wb = Workbook()
ws = wb.active
ws.title = 'snomed_train_df'
with open('/content/drive/MyDrive/snomed_train_df.csv', 'r', encoding='utf8') as f:
    for row in csv.reader(f):
      ws.append(row)
    wb.save('content/drive/MyDrive/snomed_train_df.xlsx')


**Master SNOMED CT Tabelle erstellen (Tabelle für einmalige Konzepte mit entsprechenden Synonymen. Grundlage ist die Tabelle aus dem SNOMED CT Releasepackage)**

In [ ]:
code = {}
synonym = []
for index, row in df.iterrows():
  if row['cui'] in code:
    code[row['cui']]["synonym"].append(row['name'])
  elif row['description_type_ids'] == "procedure" or row['description_type_ids'] == "body structure":
    code[row['cui']] = {
        "name": row['name'],
        "synonym": []
        }



In [ ]:
print(len(code))
c = []
n = []
s = []
for k, v in code.items():
  c.append(k)
  n.append(v["name"])
  s.append(v["synonym"])

cd = { 
    'Code': c,
    'name': n,
    'synonyms': s  
}
df = pd.DataFrame(cd)
print(df["Code"])

Bereinigen des Hauptkonzeptnames 

In [ ]:
for index, row in df.iterrows(): 
 text_without_proc = row['Text'].replace("(procedure)", "")
 text_without_body_strac = text_without_proc.replace("(body structure)", "")
 text_without_disorder = text_without_body_strac.replace("(disorder)", "")
 row['Text'] = text_without_disorder

print(df)

In [ ]:
excel_path = '/content/drive/MyDrive/master_data.xlsx'
sheet_name = "snomed_master"
print(df)
with pd.ExcelWriter(excel_path) as writer:  
    df.to_excel(writer, sheet_name=sheet_name, index=False)

**Daten aus den zusammengeführten Weiterbildungstabellen SIWF (bereits parametriert) generieren**
  

SIWF Tabellen einlesen, Quries auseinander nehmen und neues "erweitertes" Dataframe erstellen (Beschreibung, Code, Label)

In [ ]:
excel_path = '/content/drive/MyDrive/dataset.xlsx'
sheet_name = "siwf_cleaned_translated"

df = pd.read_excel(excel_path, sheet_name)
#print(df)

codeIDs = []
for x in df['Code']:
  #print(x)
  if (type(x) is str):
    id_lst = re.findall(r'\d+', x)
    #print(id_lst)
    codeIDs.append(id_lst)
  else:
    codeIDs.append([])
#print(codeIDs)
df['CodeIDs'] = codeIDs

text = []
code = []
label = []
for i in range(len(df)):
    for c in df.iloc[i,5]:
      text.append(df.iloc[i,1])
      code.append(c)
      label.append(df.iloc[i,0])
      
siwf_train_df = pd.DataFrame({TEXT_COLUMN : text,
                         SNOMED_CT_CODE_COLUMN : code,
                         LABEL_COLUMN : label
                         })
    
print(siwf_train_df)

In [ ]:
excel_path = '/content/drive/MyDrive/train_data.xlsx'
siwf_sheet_name = "siwf_train_df"
print(siwf_train_df)
with pd.ExcelWriter(excel_path) as writer:  
    siwf_train_df.to_excel(writer, sheet_name=siwf_sheet_name, index=False)

**Maximale Länge der tokenisierten Texte überprüfen** 
(Nach dem Anreichern von Texten mit Kontext kann so überprüft werden, wie lange die Sätze sind. => Problem ist, dass BERT beim tokenisieren ein eigenes Stemming macht und entsprechend Wörter mehrere Tokens sein können) Als Output der Methode wird die länge des längsten Satzes im DF ausgegeben. 

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2')

max_len = 0

sentences = df.Text.values

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)